In [47]:
import numpy as np
import pymatgen.symmetry.analyzer as analyzer
from pymatgen.ext.matproj import MPRester
import pymatgen.core.lattice 
import pymatgen.core.operations

with MPRester("v5zOFTlhqtoyEiw1") as m:
    material_id = "mp-753260"
    # Structure for material id
    structure = m.get_structure_by_material_id(material_id)
    structure.to(filename="LaAgO2_computed.cif")#computes the computed version
    
    #0 define new structure to obtain a=b=c and alpha=beta=gamma ainsi que le symmetrized et le conventional car je ne sais pas 
    #lequel je vais utiliser
    new_structure =  analyzer.SpacegroupAnalyzer(structure).get_primitive_standard_structure()
    new_structure2 = analyzer.SpacegroupAnalyzer(structure).get_symmetrized_structure()
    new_structure3 = analyzer.SpacegroupAnalyzer(structure).get_conventional_standard_structure()
    
    #a partir d 'ici je choisi de travailler sur la partie conventional_standard_structure 
    #car c'est la seule qui marche sur Jmol Cristal Symmetry Explorer
    
    #1 symmetry operations of the point group and space group
    
    SGclass3 = analyzer.SpacegroupAnalyzer(new_structure3) #symmetrized
        
    #print("point group operations  : ",SGclass2.get_point_group_operations(cartesian=False))
    PGO = SGclass3.get_point_group_operations(cartesian=False)
    
    

In [48]:
#2 get the 3 first atoms from the symmetrized structure
atomes = new_structure.sites[0:3] 
position = np.array([atomes[0].coords,atomes[1].coords,atomes[2].coords])
print(atomes)
#print('the position of the atoms:\n',position)


[PeriodicSite: La (9.2022, -0.0000, 1.6357) [0.5000, 0.5000, 0.5000], PeriodicSite: Ag (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000], PeriodicSite: O (2.0404, 0.0000, 0.3627) [0.1109, 0.1109, 0.1109]]


In [49]:
#3 get the rotation matrix for different symmetry operations 3==>1, 8==>2, 11==>3
rot_m = np.array([PGO[2].rotation_matrix,PGO[7].rotation_matrix,PGO[10].rotation_matrix])
#print(rot_m)

In [50]:
#4 calculate the new positions of the atoms with the rotation matrix of each symmetry
new_positions = rot_m @ position.T

#each vector contains 3 matrixes / each matrix contains the new postion of the atom ( a colmun per atom)
#no need to take into account translator vector as it is null
#print(new_positions)


In [51]:
#5 make a nice print to summarize everything
print("space group symbol \n : ",SGclass3.get_space_group_operations())
print('\n')
print("base vectors (conventional ) : ")
print("length of the base vectors of the direct lattice (a,b,c): \n",new_structure3.lattice.abc)
print("angles (alpha,beta,gamma) of the base vectors of the direct lattice: \n",new_structure3.lattice.angles)
print('\n')
print("initial position of  the atoms :")

for i in range(len(position)) :
    print(atomes[i].specie)
    print( " initial position: " ,position[i])
print('\n')
print('#######################################################################################')
for j in range(len(rot_m)):
    print("Symmetry operation ",j+1)
    print("rotation matrix : \n")
    print(rot_m[j])
    print('\n')
    for k in range(len(position)):
        print("new position " + str(atomes[k].specie)+" : ")
        print(new_positions[j][:,k])
    print('\n')
    print('#######################################################################################')


space group symbol 
 :  R-3m (166) spacegroup


base vectors (conventional ) : 
length of the base vectors of the direct lattice (a,b,c): 
 (3.8366436724669115, 3.8366436724669115, 18.692813040000054)
angles (alpha,beta,gamma) of the base vectors of the direct lattice: 
 (90.0, 90.0, 119.99999999999999)


initial position of  the atoms :
La
 initial position:  [ 9.20216690e+00 -5.55111512e-16  1.63567698e+00]
Ag
 initial position:  [0. 0. 0.]
O
 initial position:  [2.04035003 0.         0.3626704 ]


#######################################################################################
Symmetry operation  1
rotation matrix : 

[[ 0. -1.  0.]
 [ 1. -1.  0.]
 [ 0.  0.  1.]]


new position La : 
[5.55111512e-16 9.20216690e+00 1.63567698e+00]
new position Ag : 
[0. 0. 0.]
new position O : 
[0.         2.04035003 0.3626704 ]


#######################################################################################
Symmetry operation  2
rotation matrix : 

[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.